In [1]:
# The molecular volume of acetonitrile is 
Vm_cm3_per_mol = 52.5 # cm^3/mol
nm3_per_cm3 = (1.0e7)**3
Vm_nm3_per_mol = Vm_cm3_per_mol*nm3_per_cm3

# Avogadros number
N_A = 6.022e23  # (particles)  per mol = mol^{-1}

nparticles = 320
nparticles_in_moles = nparticles/N_A  # mol
V_in_nm3 = Vm_nm3_per_mol*nparticles_in_moles

box_length_in_nm = (V_in_nm3**(1/3))
print(f'box_length_in_nm for {nparticles} molecules of acetonitrile: {box_length_in_nm}') 


box_length_in_nm for 320 molecules of acetonitrile: 3.032886631838656


In [2]:
import gromacs
import os

In [3]:
os.system('gmx editconf -f acetonitrile_GMX.gro -box 3.1 3.1 3.1 -o acetonitrile_single.gro')

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 6 atoms
Volume: 91.4084 nm^3, corresponds to roughly 41100 electrons
No velocities found
    system size :  0.202  0.178  0.318 (nm)
    center      :  0.079 -0.005 -0.056 (nm)
    box vectors :  4.042  3.558  6.356 (nm)
    box angles  :  90.00  90.00  90.00 (degrees)
    box volume  :  91.41               (nm^3)
    shift       :  1.471  1.555  1.606 (nm)
new center      :  1.550  1.550  1.550 (nm)
new box vectors :  3.100  3.100  3.100 (nm)
new box angles  :  90.00  90.00  90.00 (degrees)
new box volume  :  29.79               (nm^3)


                     :-) GROMACS - gmx editconf, 2020.5 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
    

0

In [4]:
os.system('gmx insert-molecules -f acetonitrile_single.gro -ci acetonitrile_GMX.gro -box 3.1 3.1 3.1 -nmol 319 -try 20000 -o acetonitrile_to_be_equilibrated.gro')


         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary.

NOTE: From version 5.0 gmx insert-molecules uses the Van der Waals radii
from the source below. This means the results may be different
compared to previous GROMACS versions.

++++ PLEASE READ AND CITE THE FOLLOWING REFERENCE ++++
A. Bondi
van der Waals Volumes and Radii
J. Phys. Chem. 68 (1964) pp. 441-451
-------- -------- --- Thank You --- -------- --------



                 :-) GROMACS - gmx insert-molecules, 2020.5 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:


0

In [5]:
import sys
sys.path.append('../../')
from mdp_helpers import *

workdir = './work'
if not os.path.exists(workdir):
    os.mkdir(workdir)

    
import gromacs

In [6]:
## Minimize the system
em_mdpfile = f'{workdir}/em.mdp'
write_gromacs_min_mdp(em_mdpfile)

em_tprfile = f'{workdir}/em.tpr'
em_grofile = f'{workdir}/em.gro'
em_logfile = f'{workdir}/em.log'
em_trrfile = f'{workdir}/em.trr'
em_edrfile = f'{workdir}/em.edr'

input_grofile = 'acetonitrile_to_be_equilibrated.gro'
input_topfile = 'acetonitrile_GMX.top'

In [7]:
import subprocess

# Run GROMACS grompp command using subprocess
subprocess.run(['gmx', 'grompp', '-f', em_mdpfile, '-c', input_grofile, '-p', input_topfile, '-o', em_tprfile])

# Run GROMACS mdrun command using subprocess
subprocess.run(['gmx', 'mdrun', '-v', '-s', em_tprfile, '-o', em_trrfile, '-c', em_grofile, '-e', em_edrfile, '-g', em_logfile])

                      :-) GROMACS - gmx grompp, 2020.5 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
     

Analysing residue names:
There are:   320      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 20x20x20, spacing 0.155 0.155 0.155
This run will generate roughly 1 Mb of data



Steepest Descents:
   Tolerance (Fmax)   =  1.00000e+01
   Number of steps    =       100000
Step=    0, Dmax= 1.0e-02 nm, Epot=  2.02537e+05 Fmax= 3.18257e+05, atom= 1706
Step=    1, Dmax= 1.0e-02 nm, Epot=  1.54244e+05 Fmax= 1.23425e+05, atom= 291
Step=    2, Dmax= 1.2e-02 nm, Epot=  1.00153e+05 Fmax= 5.55568e+04, atom= 1599
Step=    3, Dmax= 1.4e-02 nm, Epot=  5.85432e+04 Fmax= 2.30685e+04, atom= 1905
Step=    4, Dmax= 1.7e-02 nm, Epot=  2.99592e+04 Fmax= 9.24607e+03, atom= 1905
Step=    5, Dmax= 2.1e-02 nm, Epot=  1.62684e+04 Fmax= 1.23320e+04, atom= 1707
Step=    7, Dmax= 1.2e-02 nm, Epot=  1.29702e+04 Fmax= 1.23436e+04, atom= 1706
Step=    8, Dmax= 1.5e-02 nm, Epot=  1.22206e+04 Fmax= 1.36410e+04, atom= 1706
Step=   10, Dmax= 9.0e-03 nm, Epot=  7.64337e+03 Fmax= 4.02919e+03, atom= 2126
Step=   11, Dmax= 1.1e-02 nm, Epot=  7.13831e+03 Fmax= 1.18434e+04, atom= 1706
Step=   12, Dmax= 1.3e-02 nm, Epot=  6.28687e+03 Fmax= 1.28428e+04, atom= 212
Step=   14, Dmax= 7.7e-03 nm, Epot=  3.

CompletedProcess(args=['gmx', 'mdrun', '-v', '-s', './work/em.tpr', '-o', './work/em.trr', '-c', './work/em.gro', '-e', './work/em.edr', '-g', './work/em.log'], returncode=0)

In [8]:
# NvT equilibrate the system

def write_gromacs_nvt_equil_mdp_for_acetonitrile(filename):
    ''' writes a gromacs mdp file for equilibration'''
    with open(filename, 'w') as f:
        f.write(f'''define                 = -DPOSRE          ; position restrain the protein
; Run parameters
integrator             = md                          ; leap-frog integrator
nsteps                 = 100000                      ; 2 * 100000 = 200 ps
dt                     = 0.002                       ; 2 fs
; Output control
nstxout                = 1000                        ; save coordinates every 2 ps
nstvout                = 1000                        ; save velocities every 2 ps
nstenergy              = 1000                        ; save energies every 2 ps
nstlog                 = 1000                        ; update log file every 2 ps
; Bond parameters
continuation           = no                          ; Initial simulation
constraint_algorithm   = lincs                       ; holonomic constraints
constraints            = h-bonds                     ; all bonds (even heavy atom-H bonds) constrained
lincs_iter             = 1                           ; accuracy of LINCS
lincs_order            = 4                           ; also related to accuracy
; Neighborsearching
ns_type                = grid                        ; search neighboring grid cels
nstlist                = 10                          ; 20 fs
rlist                  = 1.4                         ; short-range neighborlist cutoff (in nm)
rcoulomb               = 1.4                         ; short-range electrostatic cutoff (in nm)
rvdw                   = 1.4                         ; short-range van der Waals cutoff (in nm)
; Electrostatics
coulombtype            = PME                         ; Particle Mesh Ewald for long-range electrostatics
pme_order              = 4                           ; cubic interpolation
fourierspacing         = 0.16                        ; grid spacing for FFT
; Temperature coupling is on
tcoupl                 = V-rescale                   ; Weak coupling for initial equilibration
tc-grps                = System                      ; two coupling groups - more accurate
tau_t                  = 0.1                         ; time constant, in ps
ref_t                  = 298.15                      ; reference temperature, one for each group, in K
; Pressure coupling is on
pcoupl                 = Berendsen                   ; Pressure coupling on in NPT, also weak coupling
pcoupltype             = isotropic                   ; uniform scaling of x-y-z box vectors
tau_p                  = 2.0                         ; time constant, in ps
ref_p                  = 1.0                         ; reference pressure (in bar)
compressibility        = 4.5e-5                      ; isothermal compressibility, bar^-1
refcoord_scaling       = com
; Periodic boundary conditions
pbc                    = xyz                         ; 3-D PBC
; Dispersion correction
DispCorr               = EnerPres                    ; account for cut-off vdW scheme
; Velocity generationd
gen_vel                = yes                         ; Velocity generation is on
gen_temp               = 298.15                      ; temperature for velocity generation
gen_seed               = -1                          ; random seed
; These options remove COM motion of the system
nstcomm                = 10
comm-mode              = Linear
comm-grps              = System''')
    f.close()

In [9]:
## Equilibrate the system
nvt_mdpfile = f'{workdir}/nvt.mdp'
write_gromacs_nvt_equil_mdp_for_acetonitrile(nvt_mdpfile)

nvt_tprfile = f'{workdir}/nvt.tpr'
nvt_grofile = f'{workdir}/nvt.gro'
nvt_logfile = f'{workdir}/nvt.log'
nvt_xtcfile = f'{workdir}/nvt.xtc'
nvt_trrfile = f'{workdir}/nvt.trr'
nvt_edrfile = f'{workdir}/nvt.edr'
nvt_cptfile = f'{workdir}/nvt.cpt'

In [10]:
# Run GROMACS grompp command using subprocess
subprocess.run(['gmx', 'grompp', '-f', nvt_mdpfile, '-c', em_grofile, '-p', input_topfile, '-o', nvt_tprfile, '-maxwarn', '1'])

# Run GROMACS mdrun command using subprocess
subprocess.run(['gmx', 'mdrun', '-v', '-s', nvt_tprfile, '-o', nvt_trrfile, '-x', nvt_xtcfile, '-c', nvt_grofile, '-e', nvt_edrfile, '-g', nvt_logfile, '-cpo', nvt_cptfile])

                      :-) GROMACS - gmx grompp, 2020.5 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
     

turning H bonds into constraints...
Analysing residue names:
There are:   320      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 298.15 K
Calculated rlist for 1x1 atom pair-list as 1.400 nm, buffer size 0.000 nm
Set rlist, assuming 4x4 atom pair-list, to 1.400 nm, buffer size 0.000 nm
Note that mdrun will redetermine rlist based on the actual pair-list setup
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 20x20x20, spacing 0.155 0.155 0.155
This run will generate roughly 5 Mb of data


starting mdrun 'smiles_molecule'
100000 steps,    200.0 ps.
step 99900, remaining wall clock time:     0 s          imb F  4% imb F  4% b F 11% vol 0.95  imb F  6% vol 0.96! imb F  4% vol 0.94  imb F  4% vol 0.93  imb F  7% vol 0.90  imb F  6% vol 0.87  imb F  7% vol 0.93  imb F 10% imb F  5% imb F 10% imb F  6% imb F  4% imb F 12% imb F  4% imb F 25% imb F  7% imb F 11% imb F 10% imb F  7% imb F  7% imb F  7% imb F  4% imb F  4% imb F  2% imb F  7% imb F  6% imb F  2% imb F  5% imb F  5% imb F  6% imb F  4% imb F  8% imb F  4% imb F  4% imb F  4% imb F  6% imb F  4% imb F  4% imb F  4% imb F  5% imb F 14% imb F  6% imb F  6% imb F  5% imb F  9% imb F 11% imb F 10% imb F  6% imb F  2% imb F  3% imb F  4% imb F  3% imb F  3% imb F  5% imb F 12% imb F  3% imb F  5% imb F  4% imb F 11% imb F  8% imb F  7% imb F  4% imb F 11% imb F  5% imb F  9% imb F  4% imb F  5% imb F  7% imb F 12% imb F  4% imb F  4% imb F 12% imb F 16% imb F  6% imb F 18% imb F  5% imb F  9% vol 0.89! imb F  5% vol 0.

CompletedProcess(args=['gmx', 'mdrun', '-v', '-s', './work/nvt.tpr', '-o', './work/nvt.trr', '-x', './work/nvt.xtc', '-c', './work/nvt.gro', '-e', './work/nvt.edr', '-g', './work/nvt.log', '-cpo', './work/nvt.cpt'], returncode=0)

In [11]:
def write_gromacs_npt_equil_mdp_for_acetonitrile(filename):
    ''' writes a gromacs mdp file for equilibration'''
    with open(filename, 'w') as f:
        f.write(f'''define                 = -DPOSRE          ; position restrain the protein
; Run parameters
integrator             = md                          ; leap-frog integrator
nsteps                 = 100000                      ; 2 * 100000 = 200 ps
dt                     = 0.002                       ; 2 fs
; Output control
nstxout                = 1000                        ; save coordinates every 2 ps
nstvout                = 1000                        ; save velocities every 2 ps
nstenergy              = 1000                        ; save energies every 2 ps
nstlog                 = 1000                        ; update log file every 2 ps
; Bond parameters
continuation           = no                          ; Initial simulation
constraint_algorithm   = lincs                       ; holonomic constraints
constraints            = h-bonds                     ; all bonds (even heavy atom-H bonds) constrained
lincs_iter             = 1                           ; accuracy of LINCS
lincs_order            = 4                           ; also related to accuracy
; Neighborsearching
ns_type                = grid                        ; search neighboring grid cels
nstlist                = 10                          ; 20 fs
rlist                  = 1.4                         ; short-range neighborlist cutoff (in nm)
rcoulomb               = 1.4                         ; short-range electrostatic cutoff (in nm)
rvdw                   = 1.4                         ; short-range van der Waals cutoff (in nm)
; Electrostatics
coulombtype            = PME                         ; Particle Mesh Ewald for long-range electrostatics
pme_order              = 4                           ; cubic interpolation
fourierspacing         = 0.16                        ; grid spacing for FFT
; Temperature coupling is on
tcoupl                 = V-rescale                   ; Weak coupling for initial equilibration
tc-grps                = System                      ; two coupling groups - more accurate
tau_t                  = 0.1                         ; time constant, in ps
ref_t                  = 298.15                      ; reference temperature, one for each group, in K
; Pressure coupling is on
pcoupl                 = Berendsen                   ; Pressure coupling on in NPT, also weak coupling
pcoupltype             = isotropic                   ; uniform scaling of x-y-z box vectors
tau_p                  = 2.0                         ; time constant, in ps
ref_p                  = 1.0                         ; reference pressure (in bar)
compressibility        = 4.5e-5                      ; isothermal compressibility, bar^-1
refcoord_scaling       = com
; Periodic boundary conditions
pbc                    = xyz                         ; 3-D PBC
; Dispersion correction
DispCorr               = EnerPres                    ; account for cut-off vdW scheme
; Velocity generationd
gen_vel                = yes                         ; Velocity generation is on
gen_temp               = 298.15                      ; temperature for velocity generation
gen_seed               = -1                          ; random seed
; These options remove COM motion of the system
nstcomm                = 10
comm-mode              = Linear
comm-grps              = System''')
    f.close()

In [12]:
## Equilibrate the system
workdir = './work'
npt_mdpfile = f'{workdir}/npt.mdp'
write_gromacs_npt_equil_mdp_for_acetonitrile(npt_mdpfile)

nvt_tprfile = f'{workdir}/nvt.tpr'
nvt_grofile = f'{workdir}/nvt.gro'
nvt_logfile = f'{workdir}/nvt.log'
nvt_xtcfile = f'{workdir}/nvt.xtc'
nvt_trrfile = f'{workdir}/nvt.trr'
nvt_edrfile = f'{workdir}/nvt.edr'
nvt_cptfile = f'{workdir}/nvt.cpt'
input_topfile = 'acetonitrile_GMX.top'

npt_tprfile = f'{workdir}/npt.tpr'
npt_grofile = f'{workdir}/npt.gro'
npt_logfile = f'{workdir}/npt.log'
npt_xtcfile = f'{workdir}/npt.xtc'
npt_trrfile = f'{workdir}/npt.trr'
npt_edrfile = f'{workdir}/npt.edr'
npt_cptfile = f'{workdir}/npt.cpt'


In [13]:
subprocess.run(['gmx', 'grompp', '-f', npt_mdpfile, '-c', nvt_grofile, '-p', input_topfile, '-o', npt_tprfile, '-maxwarn', '1'])
subprocess.run(['gmx', 'mdrun', '-v', '-s', npt_tprfile, '-o', npt_trrfile, '-x', npt_xtcfile, '-c', npt_grofile, '-e', npt_edrfile, '-g', npt_logfile, '-cpo', npt_cptfile])


                      :-) GROMACS - gmx grompp, 2020.5 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
     

turning H bonds into constraints...
Analysing residue names:
There are:   320      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 298.15 K
Calculated rlist for 1x1 atom pair-list as 1.400 nm, buffer size 0.000 nm
Set rlist, assuming 4x4 atom pair-list, to 1.400 nm, buffer size 0.000 nm
Note that mdrun will redetermine rlist based on the actual pair-list setup
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 20x20x20, spacing 0.152 0.152 0.152
This run will generate roughly 5 Mb of data


starting mdrun 'smiles_molecule'
100000 steps,    200.0 ps.
step 99900, remaining wall clock time:     0 s          imb F  6% imb F  3% b F  9% vol 0.97  imb F  6% vol 0.97  imb F  5% vol 0.97  imb F  8% vol 0.94  imb F 13% vol 0.92  imb F  4% vol 0.90  imb F  8% vol 0.93  imb F  4% vol 0.93  imb F  6% imb F  9% imb F 18% imb F 10% imb F  5% imb F  5% imb F 14% imb F  3% imb F  5% imb F  7% imb F  9% imb F  6% imb F  7% imb F 13% imb F  5% imb F  4% imb F  5% imb F 10% imb F  6% imb F  6% imb F  4% imb F 13% imb F  5% imb F  6% imb F  2% imb F  7% imb F  6% imb F  9% imb F  4% imb F 86% imb F  9% imb F  4% imb F  3% imb F  3% imb F  8% imb F  4% imb F  3% imb F  4% imb F 10% imb F  5% imb F  3% imb F  2% imb F  6% imb F  6% imb F  3% imb F  4% imb F  7% imb F  3% imb F  5% imb F 10% imb F  9% imb F  6% imb F  5% imb F  6% imb F  7% imb F  7% imb F  4% imb F  4% imb F  4% imb F  4% imb F  9% imb F  6% imb F  5% imb F  2% imb F 11% imb F  9% imb F  7% imb F  5% imb F  8% imb F  4% vol 0.

CompletedProcess(args=['gmx', 'mdrun', '-v', '-s', './work/npt.tpr', '-o', './work/npt.trr', '-x', './work/npt.xtc', '-c', './work/npt.gro', '-e', './work/npt.edr', '-g', './work/npt.log', '-cpo', './work/npt.cpt'], returncode=0)

In [14]:
import shutil

# Copy the npt.gro file to the desired location
source_file = 'work/npt.gro'
destination_file = 'acetonitrile_320_box.gro'

# Use shutil.copy to copy the file
shutil.copy(source_file, destination_file)

# Verify the file copy
if os.path.exists(destination_file):
    print(f"File copied successfully to {destination_file}")
else:
    print("File copy failed.")

File copied successfully to acetonitrile_320_box.gro


In [15]:
import shutil

# Copy the npt.gro file to the desired location
source_file = 'smiles_molecule.acpype/smiles_molecule_GMX.itp'
destination_file = 'acetonitrile_320_box.itp'

# Use shutil.copy to copy the file
shutil.copy(source_file, destination_file)

# Verify the file copy
if os.path.exists(destination_file):
    print(f"File copied successfully to {destination_file}")
else:
    print("File copy failed.")

File copied successfully to acetonitrile_320_box.itp
